In [19]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
import toolz

import os

def custom(data):
    return toolz.curried.pipe(data, alt.to_json(filename='altdata/{prefix}-{hash}.{extension}') )
alt.data_transformers.register('custom', custom)
alt.data_transformers.enable('custom')

pass

In [43]:
df = pd.read_csv("dpt2020.csv", sep=";")
df.drop(df[df.preusuel == '_PRENOMS_RARES'].index, inplace=True)
df.drop(df[df.dpt == 'XX'].index, inplace=True)
df['annais'] = df['annais'].astype(int)
df['sexe'] = df['sexe'].replace({1: 'M', 2: 'F'})
df = df.rename(columns={"annais": "annee"})

In [70]:
periode_debut = 2010
periode_fin = 2015
df_filtered = df[(df['annee'] >= periode_debut) & (df['annee'] <= periode_fin)]

# Aggréger les données par prénom et sexe
df_aggregated = df_filtered.groupby(['preusuel', 'sexe'], as_index=False)['nombre'].sum()

# Sélection des 10 prénoms les plus populaires pour les garçons
top_10_boys = df_aggregated[df_aggregated['sexe'] == 'M'].nlargest(10, 'nombre')

# Sélection des 10 prénoms les plus populaires pour les filles
top_10_girls = df_aggregated[df_aggregated['sexe'] == 'F'].nlargest(10, 'nombre')

# Combinaison des deux DataFrames
df_top_10_combined = pd.concat([top_10_boys, top_10_girls])

# Création de l'histogramme avec Altair
chart = alt.Chart(df_top_10_combined).mark_bar().encode(
    x=alt.X('preusuel:N', title='Prénom', sort='-y'),
    y=alt.Y('nombre:Q', title='Nombre d\'enfants'),
    color=alt.Color('sexe:N', scale=alt.Scale(domain=['M', 'F'], range=['blue', 'pink']), title='Sexe'),
).properties(
    width=alt.Step(40)  # pour ajuster la largeur des barres
)

chart

alt.Chart(...)

In [84]:
# Aggrégation des données par prénom, sexe et année
df_aggregated = df.groupby(['preusuel', 'sexe', 'annee'], as_index=False)['nombre'].sum()

# Sélection des 10 prénoms les plus populaires pour les garçons
top_10_boys = df_aggregated[df_aggregated['sexe'] == 'M'].groupby('annee')

# Sélection des 10 prénoms les plus populaires pour les filles
top_10_girls = df_aggregated[df_aggregated['sexe'] == 'F'].groupby('annee')

# Combinaison des deux DataFrames
#df_top_10_combined = pd.concat([top_10_boys, top_10_girls])

# Sélecteur de plages d'années
slider = alt.binding_range(min=df['annee'].min(), max=df['annee'].max(), step=1, name='Année:')
select_year = alt.selection_single(fields=['annee'], bind=slider, value=[{'annee': df['annee'].max()}])

chart = alt.Chart(df_aggregated).mark_bar().encode(
    x=alt.X('preusuel:N', title='Prénom', sort="-y"),
    y=alt.Y('nombre:Q', title='Nombre d\'enfants'),
).properties(
    width=800  # pour ajuster la largeur des barres
).transform_filter(
    alt.FieldRangePredicate(field='annee', range=[1950, 1960])
).transform_filter(
    (alt.datum.rank < 10)
)

chart

C:\Users\jlare\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)